In [1]:
import pandas as pd

# Interactive map for "1000 Lakes" survey

Note the following:

 * I'm currently using the site locations from the 1995 survey
 
 * You need to "zip" `index.html` before you download it, otherwise it'll open as a new webpage (right-clicking and using Save-As on this webpage **does not** work!)
 
First, modify `quickmap` to include layers from Kartverket.

In [2]:
def quickmap(df, lon_col='longitude', lat_col='latitude', popup=None, cluster=False, 
             tiles='Stamen Terrain', aerial_imagery=False, kartverket=False, 
             layer_name='Stations'):
    """ Make an interactive map from a point dataset. Can be used with any dataframe
        containing lat/lon co-ordinates (in WGS84 decimal degrees), but primarily 
        designed to be used directly with the functions in nivapy.da.
        
    Args:
        df:            Dataframe. Must include columns for lat and lon in WGS84 decimal degrees
        lon_col:       Str. Column with longitudes
        lat_col:       Str. Column with latitudes
        popup:         Str or None. Default None. Column containing text for popup labels
        cluster:       Bool. Whether to implement marker clustering
        tiles:         Str. Basemap to use. See folium.Map for full details. Choices:        
                            - 'OpenStreetMap'
                            - 'Mapbox Bright' (Limited levels of zoom for free tiles)
                            - 'Mapbox Control Room' (Limited levels of zoom for free tiles)
                            - 'Stamen' (Terrain, Toner, and Watercolor)
                            - 'Cloudmade' (Must pass API key)
                            - 'Mapbox' (Must pass API key)
                            - 'CartoDB' (positron and dark_matter)
                            - Custom tileset by passing a Leaflet-style URL to the tiles
                              parameter e.g. http://{s}.yourtiles.com/{z}/{x}/{y}.png
        aerial_imagery: Bool. Whether to include Google satellite serial imagery as an
                        additional layer
        layer_name:     Str. Name of layer to create in "Table of Contents"                 
        
    Returns:
        Folium map
    """
    import pandas as pd
    import folium
    from folium.plugins import FastMarkerCluster
    from folium.plugins import MarkerCluster

    # Drop NaN
    df2 = df.dropna(subset=[lon_col, lat_col])

    # Get data
    if popup:
        df2 = df2[[lat_col, lon_col, popup]]
        df2[popup] = df2[popup].astype(str)
    else:
        df2 = df2[[lat_col, lon_col]]

    # Setup map
    avg_lon = df2[lon_col].mean()
    avg_lat = df2[lat_col].mean()        
    map1 = folium.Map(location=[avg_lat, avg_lon],
                      zoom_start=4,
                      tiles=tiles)

    # Add aerial imagery if desired
    if aerial_imagery:
        folium.raster_layers.TileLayer(tiles='http://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
                                       attr='google',
                                       name='Google satellite',
                                       max_zoom=20,
                                       subdomains=['mt0', 'mt1', 'mt2', 'mt3'],
                                       overlay=False,
                                       control=True).add_to(map1)
        
    if kartverket:
        folium.raster_layers.TileLayer(tiles='https://opencache.statkart.no/gatekeeper/gk/gk.open_gmaps?layers=topo4&zoom={z}&x={x}&y={y}',
                                       attr='karkverket',
                                       name='Kartverket topographic',
                                       overlay=False,
                                       control=True).add_to(map1)
        
        folium.raster_layers.TileLayer(tiles='https://opencache.statkart.no/gatekeeper/gk/gk.open_gmaps?layers=terreng_norgeskart&zoom={z}&x={x}&y={y}',
                                       attr='karkverket',
                                       name='Kartverket terrain',
                                       overlay=False,
                                       control=True).add_to(map1)
        
    # Add feature group to map
    grp = folium.FeatureGroup(name=layer_name)
    
    # Draw points
    if cluster and popup:
        locs = list(zip(df2[lat_col].values, df2[lon_col].values))
        popups = list(df2[popup].values)

        # Marker cluster with labels
        marker_cluster = MarkerCluster(locations=locs,
                                       popups=popups)        
        grp.add_child(marker_cluster)
        grp.add_to(map1)
       
    elif cluster and not popup:
        locs = list(zip(df2[lat_col].values, df2[lon_col].values))
        marker_cluster = FastMarkerCluster(data=locs)
        grp.add_child(marker_cluster)
        grp.add_to(map1)
        
    elif not cluster and popup: # Plot separate circle markers, with popup    
        for idx, row in df2.iterrows():
            marker = folium.CircleMarker(location=[row[lat_col], row[lon_col]],
                                         radius=5,
                                         weight=1,
                                         color='black',
                                         popup=folium.Popup(row[popup], parse_html=False),
                                         fill_color='red',
                                         fill_opacity=1)
            grp.add_child(marker)
        grp.add_to(map1)

    else: # Plot separate circle markers, no popup    
        for idx, row in df2.iterrows():
            marker = folium.CircleMarker(location=[row[lat_col], row[lon_col]],
                                         radius=5,
                                         weight=1,
                                         color='black',
                                         fill_color='red',
                                         fill_opacity=1)
            grp.add_child(marker)
        grp.add_to(map1)
            
    # Add layer control
    folium.LayerControl().add_to(map1)

    # Zoom to data
    xmin, xmax = df2[lon_col].min(), df2[lon_col].max()
    ymin, ymax = df2[lat_col].min(), df2[lat_col].max()
    map1.fit_bounds([[ymin, xmin], [ymax, xmax]])
        
    return map1

In [3]:
# Read data
df = pd.read_excel(r'../../1000_lakes_1995.xlsx', sheet_name='DATA')
df.head()

,Station ID,Station Code,Station name,Latitude,Longitude,Altitude
0,115,831-501,Brårvatn,59.294921,7.727118,902.0
1,116,1640-603,Tufsingen,62.612294,11.876558,781.0
2,125,2030-607,St.Valvatnet,69.694975,30.656211,162.0
3,135,1228-501,Steinavatn,59.859877,6.578783,1050.0
4,166,1018-4,Kleivsetvannet,58.119559,7.664899,93.0


In [4]:
# Map
map1 = quickmap(df, 
                lon_col='Longitude', 
                lat_col='Latitude', 
                popup='Station Code', 
                cluster=True, 
                tiles='OpenStreetMap', 
                aerial_imagery=True, 
                kartverket=True, 
                layer_name='Stations')
map1.save('index.html')
map1